In [1]:
# https://keras.io/examples/generative/wgan_gp/
#

import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(3)
import time
import sys
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import *

from tqdm import tqdm
import matplotlib.pyplot as plt
import glob
from sklearn.utils import shuffle

from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint

# example of loading the generator model and generating images
from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.models import load_model
from matplotlib import pyplot


import tensorflow as tf


from tensorflow.keras.datasets.fashion_mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import layers as L
from tensorflow.keras import initializers

from skimage import io
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob
from sklearn.utils import shuffle

In [2]:
foldername2class = {
	'0.0':  0,
	'0.05': 0,
	'0.1':  1,
	'0.15': 1,
	'0.2':  2,
	'0.25': 2,
	'0.3':  3,
	'0.35': 3,
	'0.4':  4,
	'0.45': 4,
	'0.5':  5,
	'0.55': 5,
	'0.6':  6,
	'0.65': 6,
	'0.7':  7,
	'0.75': 7,
	'0.8':  8,
	'0.85': 8,
	'0.9':  9,
	'0.95': 9,
	'1.0':  9,
}

In [3]:
def preprocess_images(images):
  images = (images - 127.5) / 127.5
  return images.astype('float32')

def generator_img(path_list: list):
    counter = 0
    max_counter = len(path_list)
    while True:
        single_path = path_list[counter]
        label_s = foldername2class[single_path.split('/')[-2]]
        image_s = preprocess_images(np.asarray(io.imread(single_path), dtype=np.float32))[..., :3]
        yield image_s, label_s
        # yield np.ones((336, 336, 3))
        counter += 1

        if counter == max_counter:
            counter = 0
            path_list = shuffle(path_list)

def train_gen():
    return generator_img(train_images_path)

In [4]:
IMG_SHAPE = (336, 336, 3)
BATCH_SIZE = 12
N_CLASSES = 10
# Size of the noise vector
noise_dim = 256

PATH_DATA = '../../expand_double_modes'
SAVE_RESULT = 'exp_result_new_code_old_arch_1'

train_images_path = []

iterator = tqdm(glob.glob(PATH_DATA + "/*"))
for single_folder in iterator:
    img_folder = shuffle(glob.glob(single_folder + '/*'))
    for indx, single_img_path in enumerate(img_folder):
        train_images_path.append(single_img_path)
iterator.close()

train_images_path = shuffle(train_images_path)

100%|██████████| 21/21 [00:00<00:00, 120.40it/s]


In [5]:
dataset = (
    tf.data.Dataset.from_generator(
        train_gen, 
        output_signature=(
            tf.TensorSpec(shape=IMG_SHAPE, dtype=np.float32),
            tf.TensorSpec(shape=(), dtype=np.int32),
        )
    )
    .shuffle(BATCH_SIZE).batch(BATCH_SIZE).prefetch(2)
)


In [6]:
train_size = len(train_images_path)

print(f'train: {train_size}')

train: 37800


In [7]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=21):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim)
    # generate labels
    labels = randint(0, n_classes, n_samples)
    return [z_input, labels]

In [8]:
#def init_weights():
#    return initializers.RandomNormal(stddev=0.02)

def init_weights():
    return None

In [9]:
class BNInferenceMode(tf.Module):
    def __init__(self, dim, eps=1e-3):
        val = np.ones(dim, dtype='float32')
        self.gamma = tf.Variable(val, name='BN/gamma')
        val = np.zeros(dim, dtype='float32')
        self.beta = tf.Variable(val, name='BN/beta')
        self.eps = eps
    
    def __call__(self, x, training=False):
        mean, var = tf.nn.moments(x, axes=[0, 1, 2], keepdims=True)
        return tf.nn.batch_normalization(
            x=x,
            mean=mean,
            variance=var,
            offset=self.beta,
            scale=self.gamma,
            variance_epsilon=self.eps,
            name='CustomBN'
        )

In [10]:
class GeneratorModel(tf.Module):
    def __init__(self, out_dim, n_classes=21, h_low=21, w_low=21):
        super().__init__()
        # Labels inputs
        self.label_layers_l = self._init_label_input_branch(n_classes, h_low, w_low)
        # Noise inputs
        self.noise_layers_l = self._init_noise_input_branch(h_low, w_low)
        # Merge layer (concat)
        self.merge = Concatenate()
        # Model layers 
        self.model_layers_l = self._init_model_branch(out_dim)
        # Final layer
        self.final_layer = self.model_layers_l[-1]

    def _init_label_input_branch(self, n_classes, h_low, w_low):
        n_nodes = h_low * w_low
        label_layers_l = [
            Embedding(n_classes, 96),
            Dense(n_nodes),
            Reshape((h_low, w_low, 1))
        ]
        return label_layers_l

    def _init_noise_input_branch(self, h_low, w_low):
        # foundation for h_low x w_low image
        n_nodes = 64 * h_low * w_low

        noise_layers_l = [
            Dense(n_nodes),
            LeakyReLU(alpha=0.2),
            Reshape((h_low, w_low, 64))
        ]
        return noise_layers_l

    def _init_model_branch(self, out_dim):
        model_layers_l = [ 
            Conv2DTranspose(
                128, (3,3), 
                strides=(2,2), padding='same',
                kernel_initializer=init_weights()
            ), # 42
            #BNInferenceMode(256),
            LeakyReLU(alpha=0.2),

            Conv2DTranspose(
                128, (3,3), 
                strides=(2,2), padding='same',
                kernel_initializer=init_weights()
            ), # 84
            #BNInferenceMode(128),
            LeakyReLU(alpha=0.2),

            Conv2DTranspose(
                128, (3,3), 
                strides=(2,2), padding='same',
                kernel_initializer=init_weights()
            ), # 168
            #BNInferenceMode(64),
            LeakyReLU(alpha=0.2),

            Conv2DTranspose(
                128, (3,3), 
                strides=(2,2), padding='same',
                kernel_initializer=init_weights()
            ), # 336
            #BNInferenceMode(64),
            LeakyReLU(alpha=0.2),
            
            Conv2D(
                out_dim, (3,3), 
                activation='tanh', padding='same',
                kernel_initializer=init_weights()
            ), # 336
        ]
        return model_layers_l
    
    @tf.function
    def __call__(self, label_i, noise_i, training=False):
        # Label branch
        for layer_label_i in self.label_layers_l:
            label_i = layer_label_i(label_i, training=training)

        # Noise branch
        for layer_noise_i in self.noise_layers_l:
            noise_i = layer_noise_i(noise_i, training=training)

        x_t = self.merge([noise_i, label_i])
        # Model branch
        for layer_model_i in self.model_layers_l:
            x_t = layer_model_i(x_t, training=training)
        
        return x_t


In [11]:
class DiscModel(tf.Module):
    def __init__(self, in_shape, out_dim, n_classes=21):
        super().__init__()
        # Labels inputs
        self.label_layers_l = self._init_label_input_branch(in_shape, n_classes)
        # Noise inputs
        self.image_layers_l = self._init_image_input_branch()
        # Merge layer (concat)
        self.merge = Concatenate()
        # Model layers 
        self.model_layers_l = self._init_model_branch(out_dim)
        # Final layer
        self.final_layer = self.model_layers_l[-1]

    def _init_label_input_branch(self, in_shape, n_classes):
        n_nodes = in_shape[0] * in_shape[1] * in_shape[2]
        label_layers_l = [
            # embedding for categorical input
            Embedding(n_classes, 96),
            # scale up to image dimensions with linear activation
            Dense(n_nodes),
            Reshape((in_shape[0], in_shape[1], in_shape[2]))
        ]
        return label_layers_l

    def _init_image_input_branch(self):
        image_layers_l = []
        return image_layers_l

    def _init_model_branch(self, out_dim):
        model_layers_l = [
            Conv2D(
                128, (3,3), 
                strides=(2,2), padding='same', 
                kernel_initializer=init_weights()
            ), # 168
            #BatchNormalization(),
            LeakyReLU(alpha=0.2),

            Conv2D(
                128, (3,3), 
                strides=(2,2), padding='same', 
                kernel_initializer=init_weights()
            ), # 84  
            #BatchNormalization(),
            LeakyReLU(alpha=0.2),

            Conv2D(
                128, (3,3), 
                strides=(2,2), padding='same', 
                kernel_initializer=init_weights()
            ), # 42  
            #BatchNormalization(),
            LeakyReLU(alpha=0.2),

            Conv2D(
                128, (3,3), 
                strides=(2,2), padding='same', 
                kernel_initializer=init_weights()
            ), # 21  
            #BatchNormalization(),
            LeakyReLU(alpha=0.2),

            Conv2D(
                128, (3,3), 
                strides=(2,2), padding='same', 
                kernel_initializer=init_weights()
            ), # 10  
            #BatchNormalization(),
            LeakyReLU(alpha=0.2),
            
            layers.Flatten(),
            layers.Dropout(0.3),
            layers.Dense(out_dim),
        ]
        return model_layers_l
    
    @tf.function
    def __call__(self, label_i, image_i, training=False):
        # Label branch
        for layer_label_i in self.label_layers_l:
            label_i = layer_label_i(label_i, training=training)
        
        # Image branch
        for layer_image_i in self.image_layers_l:
            image_i = layer_image_i(image_i, training=training)
        
        x_t = self.merge([image_i, label_i])
        # Model branch
        for layer_model_i in self.model_layers_l:
            x_t = layer_model_i(x_t, training=training)
        return x_t


In [12]:
d_model = DiscModel(in_shape=IMG_SHAPE, out_dim=1, n_classes=N_CLASSES)

In [13]:
g_model = GeneratorModel(IMG_SHAPE[-1], n_classes=N_CLASSES) 

In [14]:
#q = g_model.model_layers_l[0]
#q.get_config()

In [15]:
#g_model(np.asarray([0]*16), np.random.randn(16, 256)).shape

In [16]:
class WGAN(tf.Module):
    def __init__(
        self,
        discriminator,
        generator,
        latent_dim,
        discriminator_extra_steps=3,
        gp_weight=10.0,
    ):
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.d_steps = discriminator_extra_steps
        self.gp_weight = gp_weight

    def compile(self, d_optimizer, g_optimizer, d_loss_fn, g_loss_fn):
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_fn = d_loss_fn
        self.g_loss_fn = g_loss_fn

    @tf.function
    def gradient_penalty(self, batch_size, real_images, fake_images, real_labels):
        """ Calculates the gradient penalty.

        This loss is calculated on an interpolated image
        and added to the discriminator loss.
        """
        # Get the interpolated image
        alpha = tf.random.normal([batch_size, 1, 1, 1], 0.0, 1.0)
        diff = fake_images - real_images
        interpolated = real_images + alpha * diff

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            # 1. Get the discriminator output for this interpolated image.
            pred = self.discriminator(label_i=real_labels, image_i=interpolated, training=True)

        # 2. Calculate the gradients w.r.t to this interpolated image.
        grads = gp_tape.gradient(pred, [interpolated, real_labels])[0]
        # 3. Calculate the norm of the gradients.
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp

    def train_step(self, real_images, real_labels):
        # Get the batch size
        batch_size = tf.shape(real_images)[0]

        # For each batch, we are going to perform the
        # following steps as laid out in the original paper:
        # 1. Train the generator and get the generator loss
        # 2. Train the discriminator and get the discriminator loss
        # 3. Calculate the gradient penalty
        # 4. Multiply this gradient penalty with a constant weight factor
        # 5. Add the gradient penalty to the discriminator loss
        # 6. Return the generator and discriminator losses as a loss dictionary

        # Train the discriminator first. The original paper recommends training
        # the discriminator for `x` more steps (typically 5) as compared to
        # one step of the generator. Here we will train it for 3 extra steps
        # as compared to 5 to reduce the training time.
        for i in range(self.d_steps):
            d_loss = self._disc_train_step(real_images, real_labels)

        # Train the generator
        # Get the latent vector
        g_loss = self._generator_train_step(batch_size)

        return {"d_loss": d_loss, "g_loss": g_loss}

    @tf.function
    def _generator_train_step(self, batch_size):
        # Train the generator
        # Get the latent vector
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_labels = tf.random.uniform([batch_size], minval=0, maxval=N_CLASSES, dtype=tf.int32)
        with tf.GradientTape() as tape:
            # Generate fake images using the generator
            generated_images = self.generator(label_i=random_labels, noise_i=random_latent_vectors, training=True)
            # Get the discriminator logits for fake images
            gen_img_logits = self.discriminator(label_i=random_labels, image_i=generated_images, training=True)
            # Calculate the generator loss
            g_loss = self.g_loss_fn(gen_img_logits)

        # Get the gradients w.r.t the generator loss
        gen_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
        # Update the weights of the generator using the generator optimizer
        self.g_optimizer.apply_gradients(
            zip(gen_gradient, self.generator.trainable_variables)
        )

        return g_loss

    @tf.function
    def _disc_train_step(self, real_images, real_labels):
        # Get the batch size
        batch_size = tf.shape(real_images)[0]

        random_latent_vectors = tf.random.normal(
            shape=(batch_size, self.latent_dim)
        )
        with tf.GradientTape() as tape:
            # Generate fake images from the latent vector
            fake_images = self.generator(label_i=real_labels, noise_i=random_latent_vectors, training=True)
            # Get the logits for the fake images
            fake_logits = self.discriminator(label_i=real_labels, image_i=fake_images, training=True)
            # Get the logits for the real images
            real_logits = self.discriminator(label_i=real_labels, image_i=real_images, training=True)

            # Calculate the discriminator loss using the fake and real image logits
            d_cost = self.d_loss_fn(real_img=real_logits, fake_img=fake_logits)
            # Calculate the gradient penalty
            gp = self.gradient_penalty(batch_size, real_images, fake_images, real_labels)
            # Add the gradient penalty to the original discriminator loss
            d_loss = d_cost + gp * self.gp_weight

        # Get the gradients w.r.t the discriminator loss
        d_gradient = tape.gradient(d_loss, self.discriminator.trainable_variables)
        # Update the weights of the discriminator using the discriminator optimizer
        self.d_optimizer.apply_gradients(
            zip(d_gradient, self.discriminator.trainable_variables)
        )

        return d_loss



In [17]:
class GANMonitor():
    def __init__(self, model, num_img=100, latent_dim=128, call_sign=None):
        self.num_img = num_img
        self.latent_dim = latent_dim
        self.model = model
        self.call_sign = call_sign

    def on_epoch_end(self, epoch, logs=None, save_path=''):
        n = int(np.sqrt(self.num_img))
        random_latent_vectors = np.random.normal(size=(self.num_img, self.latent_dim))
        random_labels = np.asarray([min(x, N_CLASSES-1)  for _ in range(10) for x in range(10)])
        generated_images = self.model(label_i=random_labels, noise_i=random_latent_vectors)
        # scale from [-1,1] to [0,1]
        generated_images = (generated_images + 1) / 2.0
        self._generate_plot(generated_images, n, os.path.join(save_path, f'{epoch}'))
    
    def _generate_plot(self, examples, n, prefix):
        # plot images
        fig = plt.figure(figsize=(12,12))
        for i in range(n * n):
            # define subplot
            plt.subplot(n, n, 1 + i)
            # turn off axis
            plt.axis('off')
            # plot raw pixel data
            plt.imshow(examples[i])
        #pyplot.show()
        fig.savefig(f'{prefix}_image.png')
        plt.close('all')
    
    def save_model(self, path_save):
        if self.call_sign is None:
            raise ValueError("call sign is not set in monitor class")
        
        tf.saved_model.save(self.model, path_save, signatures=self.call_sign)

In [18]:
import gc

class GCClearCallback:
    def on_epoch_end(self, epoch=0, logs=None):
        gc.collect()
        tf.keras.backend.clear_session()

In [19]:
# Instantiate the optimizer for both networks
# (learning_rate=0.0002, beta_1=0.5 are recommended)
generator_optimizer = keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.9
)
discriminator_optimizer = keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.9
)

# Define the loss functions for the discriminator,
# which should be (fake_loss - real_loss).
# We will add the gradient penalty later to this loss function.
def discriminator_loss(real_img, fake_img):
    real_loss = tf.reduce_mean(real_img)
    fake_loss = tf.reduce_mean(fake_img)
    return fake_loss - real_loss


# Define the loss functions for the generator.
def generator_loss(fake_img):
    return -tf.reduce_mean(fake_img)


# Set the number of epochs for trainining.
epochs = 20


# Stuf in order to save model
call = g_model.__call__.get_concrete_function(
    tf.TensorSpec((1, 1), tf.int32, name='label'), tf.TensorSpec((1, noise_dim), tf.float32, name='noise')
)

# Instantiate the customer `GANMonitor` Keras callback.
cbk = GANMonitor(
    g_model,
    num_img=100, latent_dim=noise_dim,
    call_sign=call
)
gcclear_call = GCClearCallback()
# Instantiate the WGAN model.
wgan = WGAN(
    discriminator=d_model,
    generator=g_model,
    latent_dim=noise_dim,
    discriminator_extra_steps=5, # was 3
)

# Compile the WGAN model.
wgan.compile(
    d_optimizer=discriminator_optimizer,
    g_optimizer=generator_optimizer,
    g_loss_fn=generator_loss,
    d_loss_fn=discriminator_loss,
)


In [ ]:
for ep in range(epochs):
    iteration = train_size // BATCH_SIZE
    save_path = os.path.join(SAVE_RESULT, f'ep_{ep}')
    os.makedirs(save_path, exist_ok=True)
    for i, (real_images, real_labels) in enumerate(dataset.take(iteration)):
        data_losses = wgan.train_step(real_images=real_images, real_labels=real_labels)
        print('>%d, %d/%d, d=%.3f, g=%.3f' %
            (ep+1, i+1, iteration, data_losses['d_loss'], data_losses['g_loss']))
        if i % 20 == 0:
            cbk.on_epoch_end(f'i_{i}_ep_{ep}', save_path=save_path)
        
        if i % 200 == 0:
            cbk.save_model(f'{save_path}/models/i_{i}_ep_{ep}')
    # Clear session
    # Keras iteself has some memory leaks
    # Isshue: https://github.com/tensorflow/tensorflow/issues/31312
    gcclear_call.on_epoch_end(ep)

>1, 1/3150, d=5.834, g=0.018


INFO:tensorflow:Assets written to: exp_result_new_code_old_arch_1/ep_0/models/i_0_ep_0/assets


INFO:tensorflow:Assets written to: exp_result_new_code_old_arch_1/ep_0/models/i_0_ep_0/assets


>1, 2/3150, d=-37.475, g=0.126
>1, 3/3150, d=-484.791, g=0.625
>1, 4/3150, d=-2741.657, g=-1.895
>1, 5/3150, d=-6376.444, g=-2.302
>1, 6/3150, d=-5409.868, g=-5.441
>1, 7/3150, d=-6012.000, g=-17.874
>1, 8/3150, d=-6097.774, g=-41.082
>1, 9/3150, d=-7976.119, g=-87.959
>1, 10/3150, d=-6415.321, g=-129.997
>1, 11/3150, d=-7036.142, g=-146.278
>1, 12/3150, d=-7524.279, g=-313.107
>1, 13/3150, d=-7113.515, g=-568.800
>1, 14/3150, d=-5338.821, g=-683.399
>1, 15/3150, d=-4494.506, g=-1131.811
>1, 16/3150, d=-3937.126, g=-1616.784
>1, 17/3150, d=-2722.917, g=-1649.879
>1, 18/3150, d=-2189.312, g=-2224.693
>1, 19/3150, d=-1484.534, g=-1341.177
>1, 20/3150, d=-1394.634, g=-1061.192
>1, 21/3150, d=-938.669, g=-917.601
>1, 22/3150, d=-928.171, g=-442.827
>1, 23/3150, d=-957.924, g=-177.250
>1, 24/3150, d=-888.872, g=-191.916
>1, 25/3150, d=-793.486, g=-453.666
>1, 26/3150, d=-799.860, g=-149.865
>1, 27/3150, d=-1235.133, g=-391.883
>1, 28/3150, d=-841.568, g=318.918
>1, 29/3150, d=-1211.765, g=3

INFO:tensorflow:Assets written to: exp_result_new_code_old_arch_1/ep_0/models/i_200_ep_0/assets


...

INFO:tensorflow:Assets written to: exp_result_new_code_old_arch_1/ep_1/models/i_0_ep_1/assets


>2, 2/3150, d=-186.191, g=10256.495
>2, 3/3150, d=316.709, g=7927.686
>2, 4/3150, d=-177.574, g=5863.604
>2, 5/3150, d=96.806, g=5103.390
>2, 6/3150, d=-204.298, g=5741.020
>2, 7/3150, d=-83.055, g=5959.042
>2, 8/3150, d=47.711, g=4447.706
>2, 9/3150, d=105.207, g=3495.412
>2, 10/3150, d=122.674, g=4872.021
>2, 11/3150, d=-109.445, g=2370.837
>2, 12/3150, d=-80.990, g=7576.971
>2, 13/3150, d=22.422, g=6085.958
>2, 14/3150, d=32.206, g=5849.702
>2, 15/3150, d=5.829, g=7901.631
>2, 16/3150, d=-226.900, g=7012.498
>2, 17/3150, d=-136.534, g=5540.092
>2, 18/3150, d=282.005, g=2639.133
>2, 19/3150, d=0.965, g=2153.045
>2, 20/3150, d=-120.141, g=3414.376
>2, 21/3150, d=-376.279, g=6387.789
>2, 22/3150, d=231.627, g=6826.615
>2, 23/3150, d=223.674, g=5334.568
>2, 24/3150, d=279.666, g=5739.682
>2, 25/3150, d=-230.957, g=8315.433
>2, 26/3150, d=131.201, g=8424.233
>2, 27/3150, d=-204.011, g=9898.717
>2, 28/3150, d=-221.029, g=9997.339
>2, 29/3150, d=-658.772, g=11748.421
>2, 30/3150, d=35.573,

INFO:tensorflow:Assets written to: exp_result_new_code_old_arch_1/ep_1/models/i_200_ep_1/assets


INFO:tensorflow:Assets written to: exp_result_new_code_old_arch_1/ep_1/models/i_200_ep_1/assets


>2, 202/3150, d=-113.984, g=1685.157
>2, 203/3150, d=-192.262, g=1620.842
>2, 204/3150, d=61.940, g=1611.551
>2, 205/3150, d=78.028, g=1928.636
>2, 206/3150, d=-183.240, g=1628.079
>2, 207/3150, d=-85.275, g=2396.481
>2, 208/3150, d=-36.350, g=2569.069
>2, 209/3150, d=157.049, g=1711.324
>2, 210/3150, d=31.192, g=1564.109
>2, 211/3150, d=-122.561, g=397.243
>2, 212/3150, d=45.805, g=83.685
>2, 213/3150, d=9.838, g=-20.883
>2, 214/3150, d=-0.301, g=183.701
>2, 215/3150, d=-145.013, g=308.064
>2, 216/3150, d=-169.735, g=100.413
>2, 217/3150, d=-68.647, g=275.214
>2, 218/3150, d=-113.990, g=356.773
>2, 219/3150, d=-37.221, g=313.872
>2, 220/3150, d=-5.253, g=394.625
>2, 221/3150, d=-117.604, g=74.343
>2, 222/3150, d=-6.282, g=94.941
>2, 223/3150, d=14.270, g=240.837
>2, 224/3150, d=-31.824, g=266.338
>2, 225/3150, d=53.991, g=152.487
>2, 226/3150, d=-9.191, g=350.127
>2, 227/3150, d=1.759, g=68.001
>2, 228/3150, d=-179.359, g=-169.985
>2, 229/3150, d=-103.304, g=-621.096
>2, 230/3150, d=-

In [ ]:
call = g_model.__call__.get_concrete_function(
    tf.TensorSpec((1, 1), tf.int32, name='label'), tf.TensorSpec((1, noise_dim), tf.float32, name='noise')
)

In [ ]:
tf.saved_model.save(g_model, f'{SAVE_RESULT}/models/test_save', signatures=call)

In [ ]:
model = tf.saved_model.load(f'{SAVE_RESULT}/models/test_save')

In [ ]:
random_d = np.random.randn(1, noise_dim).astype(np.float32)
random_l = np.random.randint(low=0, high=N_CLASSES, size=(1, 1)).astype(np.int32)

In [ ]:
output_np = model(random_l, random_d)

In [ ]:
plt.imshow((output_np[0]+1) / 2.0)

In [ ]:
w = g_model.label_layers_l[0].get_weights()[0]

In [ ]:
w /= np.expand_dims(np.sqrt(np.sum(w * w, axis=1)), axis=1)
h = np.dot(w, w.T)

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(h)